## Swapper model build test

In [82]:
import os
import sys
import math
from glob import glob
# 이 파일의 Directory의 절대 경로
__file__path = os.path.abspath("")
print(__file__path)
sys.path.append(__file__path)
import numpy as np
from PIL import Image

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset

# Configuration 
from omegaconf import DictConfig, OmegaConf
import hydra
config_path = "./configs"
config_name = "config.yaml"
try: hydra.initialize(version_base=None, 
                      config_path=config_path)
except: pass
cfg = hydra.compose(config_name=config_name)
print(OmegaConf.to_yaml(cfg))

import wandb

/Users/kackaobank1/Desktop/dev/FaceSwap/swappers/SmoothSwap
train:
  isTrain: false
  end_epoch: 1000
  batch_size: 512
  optimizer: adam
  weight_decay: 0.0005
  momentum: 0.9
  lr: 0.001
  lr_step: 10
  log_dir: ./logs
  checkpoint_dir: ./checkpoints
  device: cpu
id_emb:
  image_size: 224
  train_embedder: false
  network: resnet50
  emb_size: 512
  checkpoint_path: ''
generator:
  image_size: 256
  num_feature_init: 64
discriminator:
  image_size: 256
authur:
  name: conor.k
  version: 0.0.1



/Users/kackaobank1/.pyenv/versions/3.8.10/envs/python3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


### Model Build

In [2]:
from models.id_emb import IdEmbedder
from models.generator import Generator
from models.discriminator import Discriminator

In [72]:
class FaceSwapModel(nn.Module):
    def __init__(self, G, E, D):
        super(FaceSwapModel, self).__init__()
        self.G = G
        self.D = D
        self.E = E

    def forward(self, x_src, x_tar):
        z_src = self.E(x_src)
        swap = self.G(x_tar, z_src)
        disc = self.D(swap)
        z_swap = self.E(swap)
        return z_src, swap, z_swap, disc
    
    def get_discriminant(self, x):
        return self.D(x)

    def get_id_embedding(self, x):
        return self.E(x)
    
    def get_swap_face(self, x_src, x_tar):
        temb = self.E(x_src)
        return self.G(x_tar, temb)
    
def cosine_metric(x1, x2):
    return torch.sum(x1 * x2, dim=1) / (torch.norm(x1, dim=1) * torch.norm(x2, dim=1))

class CombineLoss(nn.Module):
    """
    SmoothSwap Combined loss 
    """
    def __init__(self, lambdas=[4., 1., 1.]):
        super(CombineLoss, self).__init__()
        self.lambdas = lambdas
        cos = nn.CosineSimilarity(dim=1, eps=1e-7)

    def forward(self, x_tar, z_src, swap, z_swap, disc):
        lambdas = [4., 1., 1.]
        L_id  = torch.mean(1 - cosine_metric(z_src, z_swap))
        L_chg = torch.mean((swap - x_tar).pow(2))
#         L_adv = torch.mean(-1. * torch.log(disc))
        L_adv = 0.
        combined_loss = lambdas[0]*L_id + lambdas[1]*L_chg + lambdas[2]*L_adv
        
        return combined_loss, L_id, L_chg, L_adv

In [73]:
# fsm.requires_grad_()

In [98]:
# dataloader
class FFHQDataset(Dataset):
    def __init__(self, image_pool, transform=None):
        self.image_pool = image_pool
        self.transform = transform
        
    def __len__(self):
        return len(self.image_pool)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_src = Image.open(self.image_pool[idx])
        img_tar = Image.open(np.random.choice(self.image_pool))
        
        if self.transform:
            img_src = self.transform(img_src)
            img_tar = self.transform(img_tar)
        
        return img_src, img_tar
    
def test_FFHQDataset(image_pool):
    import matplotlib.pyplot as plt     
    ffhq = FFHQDataset(image_pool)

    for i in np.random.randint(len(ffhq), size=3):
        sample = ffhq[i]
    
        plt.subplot(1, 2, 1)
        plt.imshow(sample[0])
        plt.subplot(1, 2, 2)
        plt.imshow(sample[1])
        plt.tight_layout()
        plt.show()

In [99]:
# train_on_batch
def train_epoch(epoch,
                train_loader, 
                model, 
                loss_fn, 
                optimizer, 
                optimizer_D, 
                cuda, 
                log_interval,
                metrics):
    
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, sample in enumerate(train_loader):
        img_src, img_tar = sample
        # add target target sample
        img_src[-1] = img_tar[-1]
        
        if cuda:
            img_src = img_src.cuda()
            img_tar = img_tar.cuda()
            
        # Discriminator
        img_fake = model.get_swap_face(img_src, img_tar).detach()
        gen_logit = model.D(img_fake)
        loss_Dgen = (F.relu(torch.ones_like(gen_logit) + gen_logit)).mean()
        
        real_logit = model.D(img_tar)
        loss_Dreal = (F.relu(torch.ones_like(real_logit) - real_logit)).mean()
        loss_d = loss_Dgen + loss_Dreal
        optimizer_D.zero_grad()
        loss_d.backward()
        optimizer_D.step()
        
        # Swapper
        z_src, swap, z_swap, disc = model(img_src, img_tar)
        loss_outputs = loss_fn(img_tar, z_src, swap, z_swap, disc)        
        loss = torch.mean(loss_outputs[0])
        losses.append(loss.item())
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        for metric in metrics:
            metric(outputs, loss_outputs)
        
        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(img_src[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses))
            
            train_log = {
                "epoch" : epoch,
                "batch" : batch_idx,
                'total_l': loss_outputs[0],
                'id_l'   : loss_outputs[1],
                'chg_l'  : loss_outputs[2],
                'gan_l'  : loss_outputs[3],
            }
            wandb.log(train_log)
            
            message += "\t id_loss:{:.3f}, chg_loss:{:.3f}, gan_loss:{:.3f}".format(*loss_outputs[1:])
            
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message, end="\r")
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def detransform_output(swap):
    imagenet_std    = torch.Tensor([0.229, 0.224, 0.225]).view(3,1,1)
    imagenet_mean   = torch.Tensor([0.485, 0.456, 0.406]).view(3,1,1)
    swap = swap * imagenet_std
    swap = swap + imagenet_mean
    return swap.numpy()


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, sample in enumerate(val_loader):
            img_src, img_tar = sample
            if cuda:
                img_src = img_src.cuda()
                img_tar = img_tar.cuda()
                
            outputs = model(*data)

            loss_inputs = outputs
            loss_outputs = loss_fn(*loss_inputs)
            
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()
            
            for metric in metrics:
                metric(outputs, target, loss_outputs)
                
        img_src = detransform_output(img_src.to('cpu')).transpose(0,2,3,1)
        img_tar = detransform_output(img_tar.to('cpu')).transpose(0,2,3,1)
        swap = detransform_output(outputs[1]).transpose(0,2,3,1)
        
        table = wandb.Table(columns=["src", "tar", "swap"])
        for src, tar, swa in zip(img_src, img_tar, swap):
            table.add_data(wandb.Image(src*255), wandb.Image(tar*255), wandb.Image(swa*255))
        wandb.log({"swap_sample":table}, commit=False)
                
    return val_loss, metrics

def fit(train_loader, 
        val_loader, 
        model, 
        loss_fn, 
        optimizer,
        optimizer_D,
        scheduler, 
        n_epochs, 
        cuda, 
        log_interval,
        checkpoint_dir="./checkpoints/smoothswap",
        metrics=[],
        start_epoch=0):

    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):

        # Train stage
        train_loss, metrics = train_epoch(epoch, train_loader, model, loss_fn, optimizer, optimizer_D, cuda, log_interval, metrics)
        scheduler.step()
        
        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())
            
        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)

        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, 
                                                                                 n_epochs,
                                                                                 val_loss)
        val_log = {
            "epoch": epoch,
            "val_loss": val_loss,            
        }
        wandb.log(val_log)
        
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())
        print(message)
        state = {'epoch': epoch,
                 'model': model,
                 'optimizer': optimizer}
        filename = os.path.join(checkpoint_dir, 
                                'cpt_' + str(epoch) + '_' + "{:.4f}".format(val_loss) + '.tar')
        torch.save(state, filename)
        print("{} is saved".format(filename))
        
    wandb.finish()
        

In [94]:
data_dir = "../../data/ffhq/images1024x1024/"
image_pool = glob(data_dir+"/*/*.png")
test_FFHQDataset(image_pool)


n_epochs = 20
batch_size = 2
log_interval = 5
margin = 1.
cuda = torch.cuda.is_available()
lr = 1e-3
lr_d = 4e-3

wandb.init(
    project="FaceSwap-SmoothSwap",
    config={
        "epochs": n_epochs,
        "batch_size": batch_size,
        "log_interval":log_interval,
        "margin" : margin,
        "lr": lr,
        "lr_d": lr_d,
        })
# config = wandb.config

batch,▁
chg_l,▁
epoch,▁
gan_l,▁
id_l,▁
total_l,▁
batch,0
chg_l,0.03458
epoch,0
gan_l,0.0
id_l,0.52033


In [4]:
# model
G = Generator(cfg)
D = Discriminator(cfg.discriminator.image_size)
E = IdEmbedder(cfg)
if cfg.id_emb.checkpoint_path:
    E.load_state_dict(cfg.id_emb.checkpoint_path)
E.requires_grad_(False)
E.eval()

cuda = torch.cuda.is_available()
if cuda:
    G.cuda()
    D.cuda()
    E.cuda()
    
    
fsm = FaceSwapModel(G, E, D)
loss_fn = CombineLoss()

# Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_set = np.random.choice(image_pool, int(len(image_pool)*0.9))
test_set  = np.array(list(set(image_pool) - set(train_set)))

train_dataset = FFHQDataset(train_set, transform=transform)
test_dataset  = FFHQDataset(test_set,  transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

NameError: name 'Generator' is not defined

In [96]:
optimizer = optim.Adam(fsm.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, batch_size, gamma=0.1, last_epoch=-1)
optimizer_D= optim.Adam(fsm.D.parameters(), lr=lr_d)

In [97]:
fit(train_loader, 
    test_loader,
    fsm,
    loss_fn, 
    optimizer,
    optimizer_D,
    scheduler, 
    n_epochs, 
    cuda, 
    log_interval)

wandb.finish()

KeyboardInterrupt: 

In [100]:

wandb.finish()

batch,▁
chg_l,▁
epoch,▁
gan_l,▁
id_l,▁
total_l,▁
batch,0
chg_l,0.04549
epoch,0
gan_l,0.0
id_l,0.40552
